In [2]:
import sqlite3
import requests
import pandas as pd

server = 'https://geodienste.hamburg.de/HH_WFS_Badegewaesser?SERVICE=WFS'


In [3]:
body = """
    <GetFeature xmlns="http://www.opengis.net/wfs" service="WFS" version="1.1.0" outputFormat="application/geo+json" xsi:schemaLocation="http://www.opengis.net/wfs http://schemas.opengis.net/wfs/1.1.0/wfs.xsd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
        <Query typeName="app:badegewaesser" srsName="EPSG:4326" xmlns:zwr_public="https://geodienste.hamburg.de/HH_WFS_Badegewaesser">
        </Query>
    </GetFeature>
"""
response = requests.post(server, body)


In [4]:
response

<Response [200]>

In [5]:
response.json()

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': 'APP_BADEGEWAESSER_6',
   'geometry': {'type': 'Point',
    'coordinates': [10.02027961672954, 53.6719784818228]},
   'properties': {'name': 'Naturbad Kiwittsmoor',
    'badesaison': '01.05.2023 bis 11.09.2023',
    'hinweis': 'NNNN',
    'eg_einstufung': 3,
    'link_zu_details': 'http://www.hamburg.de/naturbad-kiwittsmoor/',
    'bemerkung_bsu': 'Die Badesaison ist beendet.'},
   'srsName': 'EPSG:4326'},
  {'type': 'Feature',
   'id': 'APP_BADEGEWAESSER_8',
   'geometry': {'type': 'Point',
    'coordinates': [10.182971799160324, 53.662124499438875]},
   'properties': {'name': 'Sommerbad Volksdorf',
    'badesaison': '01.06.2023 bis 31.08.2023',
    'hinweis': 'NNNN',
    'eg_einstufung': 3,
    'link_zu_details': 'http://www.hamburg.de/sommerbad-volksdorf/'},
   'srsName': 'EPSG:4326'},
  {'type': 'Feature',
   'id': 'APP_BADEGEWAESSER_9',
   'geometry': {'type': 'Point',
    'coordinates': [10.101703387034236, 

In [29]:
df = pd.json_normalize(response.json()['features'])\
        .rename(columns={
                    "properties.name": "name",
                    "geometry.coordinates": "coordinates",
            })\
            .drop(columns=[
                    'type', 'srsName', 'geometry.type', 'properties.badesaison', 'properties.hinweis', 'properties.eg_einstufung', 'properties.link_zu_details', 'properties.bemerkung_bsu'
            ])\
            .set_index("id")
df

,coordinates,name
id,,
APP_BADEGEWAESSER_6,"[10.02027961672954, 53.6719784818228]",Naturbad Kiwittsmoor
APP_BADEGEWAESSER_8,"[10.182971799160324, 53.662124499438875]",Sommerbad Volksdorf
APP_BADEGEWAESSER_9,"[10.101703387034236, 53.48687492104021]","Eichbaumsee, Badestelle Nord"
APP_BADEGEWAESSER_10,"[10.110688971267237, 53.48481486729789]","Eichbaumsee, Badestelle Ost"
APP_BADEGEWAESSER_11,"[10.165284436846633, 53.48150913531032]","Allermöher See, Nordspitze"
APP_BADEGEWAESSER_12,"[10.137108192268851, 53.51502748311504]","Boberger See, Sandstrand"
APP_BADEGEWAESSER_13,"[10.028194475425444, 53.593109466089665]",Naturbad Stadtparksee
APP_BADEGEWAESSER_14,"[10.144324034868795, 53.55898784820688]","Öjendorfer See, Badestelle Nord"
APP_BADEGEWAESSER_15,"[10.138168386957762, 53.55455385492606]","Öjendorfer See, Badestelle Süd"


In [30]:
df['alternate_name'] = df['name']
df[['long', 'lat']] = df['coordinates'].apply(pd.Series)
df = df.drop('coordinates', axis=1)
locations = df.reindex(columns=["name", "alternate_name", "long", "lat"])

In [31]:
db = sqlite3.connect("../dataset.sqlite3")
locations.to_sql("locations", db, if_exists="append")


17

In [7]:
# Measurements
body = """
    <GetFeature xmlns="http://www.opengis.net/wfs" service="WFS" version="1.1.0" outputFormat="application/geo+json" xsi:schemaLocation="http://www.opengis.net/wfs http://schemas.opengis.net/wfs/1.1.0/wfs.xsd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
        <Query typeName="app:badegewaesser_proben" srsName="EPSG:4326" xmlns:zwr_public="https://geodienste.hamburg.de/HH_WFS_Badegewaesser">
        </Query>
    </GetFeature>
"""
response = requests.post(server, body)
response.json()['features']

[{'type': 'Feature',
  'id': 'APP_BADEGEWAESSER_PROBEN_25969',
  'geometry': None,
  'properties': {'tagebuchnummer': '2023HW00269',
   'messstellennummer': 'Stps',
   'name': 'Naturbad Stadtparksee',
   'badesaison': '24.04.2023 bis 11.09.2023',
   'hinweis': 'NNNN',
   'badestellen_typ': 'Badegewaesser',
   'temperatur': '8,1',
   'datum': '20230403',
   'sichttiefe': '>2,0',
   'ph_wert': '9,0',
   'enterokokken': '<15',
   'mpn': '<15',
   'farbe': 'farblos',
   'geruch': 'geruchlos',
   'algen': 'nein',
   'bemerkung_bsu': 'keine Beanstandung',
   'icon': 'NNNN'},
  'srsName': 'EPSG:4326'},
 {'type': 'Feature',
  'id': 'APP_BADEGEWAESSER_PROBEN_25970',
  'geometry': None,
  'properties': {'tagebuchnummer': '2023HW00339',
   'messstellennummer': 'Stps',
   'name': 'Naturbad Stadtparksee',
   'badesaison': '24.04.2023 bis 11.09.2023',
   'hinweis': 'NNNN',
   'badestellen_typ': 'Badegewaesser',
   'temperatur': '14,4',
   'datum': '20230424',
   'sichttiefe': '>2,0',
   'ph_wert': '